<a href="https://colab.research.google.com/github/Raymondgwangryeol/Raymondgwangryeol/blob/main/Study/ML/PytorchTutorial/PytorchTutorial_5_Build_the_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **신경망 모델 구축하기**


---


신경망은 데이터에 대한 연산을 수행하는 다양한 계층(layer)/모듈(module)로 구성되어 있음.   
이런 신경망을 구성하는데 필요한 모든 요소들을 제공하는 namespace => ***torch.nn!***

Pytorch의 모든 모듈은 nn.Module의 하위클래스임.
<br><br>
FashionMNIST 데이터 셋의 이미지를 분류하는 신경망을 만들어보자!

In [ ]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

## **학습을 위한 장치 얻기**

가능하면 CPU보다 GPU나 MPS같은 하드웨어 가속기에서 학습하는게 더 빠르다.(대부분의 경우?)

아래 코드는 *torch.cuda*나 *torch.backends.mps*가 사용 가능한지 확인하고, 만약 안 되면 CPU를 계속 사용한다.

In [ ]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
 ) # 아 이거 i if i>2 else 0 뭐 이런 식!! 이거네!! 이거다!!
print(f"Using {device} device")


Using cuda device


## **클래스 정의하기**

신경망 모델을 nn.Module의 하위 클래스로 정의하고, ***\_\_init__*에서 신경망 계층들을 초기화**함.

nn.Module을 상속받은 모든 클래스들은 ***forward 메소드***에 **입력 데이터에 대한 연산을 구현**함.

In [ ]:
#1. nn.Module 상속받은 class 정의하기
#2. __init__함수에 flatten, 어떻게 layer 쌓을 건지 초기화함.
#3. forward함수에 __init__에서 초기화 한 것들을 입력데이터에 적용하는 코드를 작성.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

In [ ]:
#4.만든 모델 Class의 instance를 생성하고 이의 데이터를 device로 보내기(.to()메서드 사용).
model = NeuralNetwork().to(device)

#구조를 출력해 봅시다
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## **모델을 사용하기 위해 입력 데이터를 전달하기**
**forward 함수**는 nn.Module에서 일부 백그라운드 연산과 함께 **자동으로 실행됨**. (model.forward() **직접 호출하지 마십시오**)
<br><br>
모델에 입력을 전달하여 호출하면 2차원 텐서(flatten)가 반환됨. dim=0은 열 기준, dim = 1은 행 기준.
<br><br>
원시 예측값을 nn.Softmax 모듈의 인스턴스에 통과시켜 예측 확률을 얻음.

In [ ]:
X = torch.rand([1, 28, 28], device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits) #pred_probab = nn.Softmax(dim=1), pred_probab(logits)랑 같은 말인듯...?
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([4], device='cuda:0')


## **모델 계층(Layer)**

FashionMNIST 모델의 계층들을 살펴보기 위해, 28X28 크기의 이미지 3개로 구성된 미니배치를 가져와, 신경망을 통과시킬 때 무슨 일이 일어나는지 알아본다.

In [ ]:
input_image = torch.rand(3, 28, 28)
print(input_image.size())

torch.Size([3, 28, 28])


### **⚡ nn.Flatten**
다차원 배열을 **1차원 배열로 변환**.  
<br>
Default : start_dim = 1 , end_dim = -1   
<br>
0번째 차원인 배치 크기는 유지하되, 나머지 다차원 데이터를 1차원으로 줄여주는 기능을 함.    

계층을 초기화하여, 28X28의 2D 이미지를 784(28*28)픽셀 값을 갖는 연속된 배열로 변환함. (0번째 차원 배치 크기는 유지)

In [ ]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


## **⚡ nn.Linear**
공부 자료: https://velog.io/@nochesita/%EB%A0%88%EC%9D%B4%EC%96%B4-%EC%9D%B4%ED%95%B4%ED%95%98%EA%B8%B0-1-Linear

저장된 **가중치(Weight)와 편향(bias)**을 사용하여 **입력에 선형 변환(linear transformation)을 적용**하는 모듈.   

주어진 행렬의 모양을 바꾸는 연산으로, 행렬로 표현될 수 있음.   
행렬로 표현된 선형변환이 바로 Weight의 모양임.

**bias:** 예측한 결과가 정답과 일정하게 차이가 나는 정도. 이 정도를 줄이기 위해 상수를 더해 원점 이동 시킴.

In [ ]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1= layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


### **⚡ nn.Softmax**
**활성화 함수(비선형 함수)**의 한 종류. 모델의 각 분류(class)에 대한 **예측 확률을 특정한 범위로 비례하여 조정**(scale)한다.
<br><br>
Softmax함수는 Transfer function(이 예제에서는 logits)으로부터 전달된 값들이 **[0, 1]사이의 값이 되도록 정규화**하여, 출력 값들의 총합이 항상 1이 되게 한다.   

보통 **출력 노드의 활성화 함수**로 많이 사용 되는 함수이다.

In [ ]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

## **모델 매개변수**

신경망 내부의 많은 계층들은 매개변수화 되어, 가중치와 편향의 최적화를 위한 학습에 사용된다.   

nn.Module을 상속하면, 모델 객체 내부의 모든 필드들이 자동으로 추적(track)되며, ***parameters()***(layer 이름을 **제외**한 parameter값 iterator 반환), ***named_parameters()***(**layer 이름 포함** 파라미터 값 반환)로 parameter 값에 대한 iterator를 얻어 parameter에 접근할 수 있다.

In [ ]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size}| Values : {param[:2]}\n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: <built-in method size of Parameter object at 0x7b28387a99e0>| Values : tensor([[-0.0310, -0.0161, -0.0091,  ...,  0.0080,  0.0316, -0.0046],
        [-0.0138, -0.0199,  0.0206,  ...,  0.0033,  0.0116,  0.0120]],
       device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.0.bias | Size: <built-in method size of Parameter object at 0x7b28387a9a30>| Values : tensor([ 0.0257, -0.0098], device='cuda:0', grad_fn=<SliceBackward0>)

Layer: linear_relu_stack.2.weight | Size: <built-in method size of Parameter object at 0x7b28387a85e0>| Values : tensor([[-0.0361, -0.0353, -0.0342,  ...,  0.0349, -0.0022, -0